In [ ]:
from quantopian.pipeline.data.builtin import USEquityPricing
from quantopian.pipeline.factors import SimpleMovingAverage
from quantopian.research import run_pipeline
from quantopian.pipeline import Pipeline,CustomFilter
from quantopian.pipeline.factors import CustomFactor,RSI
from quantopian.pipeline.data import morningstar
from zipline import TradingAlgorithm  
from quantopian.pipeline.filters import Q1500US
from quantopian.pipeline.factors import AverageDollarVolume
import numpy as np
import talib 
import pandas as pd 


class Gap(CustomFactor):
    inputs = [USEquityPricing.open, USEquityPricing.close]
    window_length = 2
    def compute(self, today, assets, out, open_price, close):
        out[:] = open_price[-1] / close[-2] - 1 
        
class PrevClose(CustomFactor):
    inputs = [USEquityPricing.close]
    window_length = 2
    def compute(self, today, assets, out, close):
        out[:] = close[-2]
        
def make_pipeline():
    base_universe = Q1500US()
    yesterday_close = PrevClose()
    today_open = USEquityPricing.open
    dollar_volume = AverageDollarVolume(window_length=30)
    rsi = RSI()#default window_length = 15
    rsi_under_60 = rsi < 60
    ## gap = today_open / yesterday_close - 1 では出来ない．
    ## TypeError: unsupported operand type(s) for /: 'BoundColumn' and 'BoundColumn'
    gap = Gap()
    
    #ToDo この範囲を色々変えてみる．
    high_dollar_volume = dollar_volume.percentile_between(90, 100)
    pipe = Pipeline(
        
        columns = {
            'close': yesterday_close,
            'today_open': today_open.latest, 
            'dollar_volume': dollar_volume,
            'high_dollar_volume': high_dollar_volume, 
            'gap': gap, 
            'rsi': rsi, 
        },
        screen = base_universe & high_dollar_volume & rsi_under_60
        
    )
    return pipe


def check_gdgu(context, data):
    context.output = pipeline_output('pipe')
    context.security_list = context.output.index
    
    # GUGD sid_list
    context.gapdowns = context.output.sort_values(by='gap').head(5).index
    context.gapups =  context.output.sort_values(by='gap').tail(5).index

results = run_pipeline(make_pipeline(), start_date='2015-1-1', end_date='2017-1-10')

l = list()
for date in results.index.get_level_values(0).unique():
    #print('running', date)
    gap_downs = results.ix[date].sort_values(by='gap').head(5).index
    #print([sid.symbol for sid in gap_downs])
    
    df = get_pricing(gap_downs, start_date=date, end_date = date, frequency = 'minute', fields='close_price')
    df.columns = map(lambda x: x.symbol,df.columns)
    df = df.fillna(method='pad')
    l.append((df.ix[60] /df.ix[0] - 1).values )
    
    
hoge = pd.DataFrame(l, index=results.index.get_level_values(0).unique())    
hoge.cumsum().plot()
        
        
        